In [231]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#### Reading .csv files to analize data

In [232]:
master_data = pd.read_csv('data/master.csv', index_col = 'playerID', usecols = [0, 13, 14])


In [233]:
salary_data = pd.read_csv('data/salaries.csv', index_col = 'playerID', usecols = [3, 4])


In [256]:
batting_data = pd.read_csv('data/batting.csv', index_col = 'playerID', usecols = [0, 1, 5, 6, 8, 15, 18, 20])

In [235]:
batting_no_nan = batting_data.fillna(0)


In [236]:
fielding_data = pd.read_csv('data/fielding.csv', index_col = 'playerID', usecols = [0, 5])


In [237]:
name_position = master_data.merge(fielding_data, left_index=True, right_index=True)


In [238]:
name_position_salary = name_position.merge(salary_data, left_index=True, right_index=True)


In [239]:
total_info = name_position_salary.merge(batting_no_nan, left_index=True, right_index=True)


### OBP = (H + BB + HBP) / (AB + BB + HBP + SF)

H - Hits;    BB - Base on Balls;    HBP - Hit by Pitch;    SF - Sacrifice flies

In [240]:
total_info['OBP'] = (total_info['H']+ total_info['BB']+ total_info['HBP']) / (total_info['AB']+ total_info['BB']+ total_info['HBP'] + total_info['SF'])
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,G,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,,
aardsda01,David,Aardsma,P,300000,2004,11,0.0,0.0,0.0,0.0,0.0,NaN
aardsda01,David,Aardsma,P,300000,2006,45,2.0,0.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2007,25,0.0,0.0,0.0,0.0,0.0,NaN
aardsda01,David,Aardsma,P,300000,2008,47,1.0,0.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2009,73,0.0,0.0,0.0,0.0,0.0,NaN


### Filtering DataFrame 
For the data analysis there's no need to have players with no salary and players whose OBP is either 1 or 0.

In [241]:
total_info = total_info[total_info['OBP'] != 0]

In [242]:
total_info = total_info[total_info['OBP'] != 1]

In [243]:
total_info = total_info[total_info['salary'] != 0]


In [244]:
total_info = total_info.dropna()


##### We will consider players that played after 2014

In [245]:
total_info = total_info.query('yearID>2014')

In [246]:
#total_info = total_info.sort(['OBP'], ascending = False)
total_info = total_info.sort_values(by = 'OBP', ascending = False)


#### The roster doesn't include Designated Hitter (DH) and Outfielder (OF) positions

In [247]:
total_info = total_info[total_info['POS'] != 'DH']
total_info = total_info[total_info['POS'] != 'OF']


In [248]:
total_info = total_info.drop_duplicates()

In [249]:
total_info['OBP by salary'] = total_info['OBP'] / total_info['salary']
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,G,AB,H,BB,HBP,SF,OBP,OBP by salary
playerID,,,,,,,,,,,,,
burriem01,Emmanuel,Burriss,RF,410000,2015,5,3.0,2.0,2.0,0.0,0.0,0.8,0.000002
burriem01,Emmanuel,Burriss,LF,404000,2015,5,3.0,2.0,2.0,0.0,0.0,0.8,0.000002
burriem01,Emmanuel,Burriss,RF,625000,2015,5,3.0,2.0,2.0,0.0,0.0,0.8,0.000001
burriem01,Emmanuel,Burriss,RF,404000,2015,5,3.0,2.0,2.0,0.0,0.0,0.8,0.000002
burriem01,Emmanuel,Burriss,3B,625000,2015,5,3.0,2.0,2.0,0.0,0.0,0.8,0.000001


In [250]:
total_info = total_info.sort_values(by = 'OBP by salary', ascending = False)

In [251]:
total_info = total_info.drop_duplicates('salary')

In [267]:
def roster(total_info):
    roster_list = []   
    pos_list = ['1B', '2B', '3B', 'RF', 'LF', 'CF', 'P', 'C','SS']
    for pos in pos_list:
        player_position = total_info[total_info['POS'] == pos]
        roster_list.append(player_position[:1])
    return roster_list

In [268]:
player_list = roster(total_info)
final_roster = pd.concat(player_list)
final_roster

,nameFirst,nameLast,POS,salary,yearID,G,AB,H,BB,HBP,SF,OBP,OBP by salary
playerID,,,,,,,,,,,,,
ortizda01,David,Ortiz,1B,170000,2015,146,528.0,144.0,77.0,0.0,9.0,0.359935,2.117264e-06
burriem01,Emmanuel,Burriss,2B,625000,2015,5,3.0,2.0,2.0,0.0,0.0,0.800000,1.280000e-06
cabremi01,Miguel,Cabrera,3B,165574,2015,119,429.0,145.0,77.0,3.0,2.0,0.440313,2.659313e-06
burriem01,Emmanuel,Burriss,RF,404000,2015,5,3.0,2.0,2.0,0.0,0.0,0.800000,1.980198e-06
beltrca01,Carlos,Beltran,LF,200000,2015,133,478.0,132.0,45.0,2.0,6.0,0.337100,1.685499e-06
sizemgr01,Grady,Sizemore,CF,318300,2015,58,175.0,45.0,14.0,2.0,1.0,0.317708,9.981412e-07
masteju01,Justin,Masterson,P,415500,2015,18,3.0,2.0,0.0,0.0,0.0,0.666667,1.604493e-06
pierzaj01,A. J.,Pierzynski,C,210000,2015,113,407.0,122.0,19.0,7.0,3.0,0.339450,1.616426e-06
wrighda03,David,Wright,SS,322500,2015,38,152.0,44.0,22.0,0.0,0.0,0.379310,1.176156e-06


#### Below is the minimum salary and maximum OBP for each position

In [269]:
total_info.groupby('POS').salary.min()

POS
1B    170000
2B    307500
3B    165574
C     210000
CF    305000
LF    200000
P     150000
RF    302500
SS    300900
Name: salary, dtype: int64

In [270]:
total_info.groupby('POS').OBP.max()

POS
1B    0.800000
2B    0.800000
3B    0.440313
C     0.444444
CF    0.401460
LF    0.460245
P     0.666667
RF    0.800000
SS    0.390244
Name: OBP, dtype: float64